<a href="https://colab.research.google.com/github/tg-bomze/Face-Depixelizer/blob/master/Face_Depixelizer_Eng.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<b><font color="black" size="+4">Face Depixelizer</font></b>

Given a low-resolution input image, Face Depixelizer searches the outputs of a generative model (here, StyleGAN) for high-resolution images that are perceptually realistic and downscale correctly.

<b><font color="black" size="+2">Based on:</font></b>

**GitHub repository**: [PULSE](https://github.com/adamian98/pulse)

Article: [PULSE: Self-Supervised Photo Upsampling via Latent Space Exploration of Generative Models](https://arxiv.org/abs/2003.03808)

Creators: **[Alex Damian](https://github.com/adamian98), [Sachit Menon](mailto:sachit.menon@duke.edu).**

<b><font color="black" size="+2">Colab created by:</font></b>

GitHub: [@tg-bomze](https://github.com/tg-bomze),
Telegram: [@bomze](https://t.me/bomze),
Twitter: [@tg_bomze](https://twitter.com/tg_bomze).

---
##### <font color='red'>Currently using Google Drive to store model weights and it has a daily cap on downloads, therefore, you may receive an error message saying "Google Drive Quota Exceeded" or "No such file or directory: '/content/pulse/runs/face.png'". If you are experiencing this error please try again later in the day or come back tomorrow.</font>

```
To get started, click on the button (where the red arrow indicates). After clicking, wait until the execution is complete.
```



In [ ]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> Let's ROCK!</font></b>
#@markdown **After starting this block, you will need to scroll down page and upload a pixel square photo into which the whole human head fits. Neural network works best on images where people are directly facing the camera. Example:**

#@markdown ![example](https://github.com/tg-bomze/Face-Depixelizer/raw/master/example.jpg)

#@markdown *You can crop the photo [HERE](https://www.iloveimg.com/crop-image)*

#@markdown ---
import torch
import torchvision
from pathlib import Path
if not Path("PULSE.py").exists():
  if Path("pulse").exists():
    %cd /content/pulse
  else:
    !git clone https://github.com/adamian98/pulse
    %cd /content/pulse
    !mkdir input/
    toPIL = torchvision.transforms.ToPILImage()
    toTensor = torchvision.transforms.ToTensor()
    from bicubic import BicubicDownSample
    D = BicubicDownSample(factor=1)

import os
from io import BytesIO
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
from PULSE import PULSE
from google.colab import files
from bicubic import BicubicDownSample
from IPython import display
from IPython.display import display
from IPython.display import clear_output
import numpy as np
from drive import open_url
from mpl_toolkits.axes_grid1 import ImageGrid
%matplotlib inline

#@markdown ## Basic settings:
#@markdown ##### *If you have already uploaded a photo and just want to experiment with the settings, then uncheck the following checkbox*:
upload_new_photo = True #@param {type:"boolean"}


if upload_new_photo == True:
  !rm -rf /content/pulse/input/face.png
  clear_output()
  uploaded = files.upload()
  for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))
  os.rename(fn, fn.replace(" ", ""))
  fn = fn.replace(" ", "")

  if(len(uploaded.keys())!=1): raise Exception("You need to upload only one image.")

  face = Image.open(fn)
  face = face.resize((1024, 1024), Image.ANTIALIAS)
  face = face.convert('RGB')
  face_name = 'face.png'
  face.save(face_name)
  %cp $face_name /content/pulse/input/

  images = []
  imagesHR = []
  imagesHR.append(face)
  face = toPIL(D(toTensor(face).unsqueeze(0).cuda()).cpu().detach().clamp(0,1)[0])
  images.append(face)

#@markdown ---
#@markdown ## Advanced settings:
#@markdown ##### *If you want to make a more accurate result, then modify the following* **DEFAULT** *variables*:

input_dir = '/content/pulse/input/'
output_dir = '/content/pulse/runs/'
seed = 100 #@param {type:"integer"}
epsilon = 0.02 #@param {type:"slider", min:0.01, max:0.03, step:0.01}
noise_type = 'trainable'  #@param ['zero', 'fixed', 'trainable']
optimizer = 'adam'  #@param ['sgd', 'adam','sgdm', 'adamax']
learning_rate = 0.4 #@param {type:"slider", min:0, max:1, step:0.05}
learning_rate_schedule = 'linear1cycledrop'  #@param ['fixed', 'linear1cycle', 'linear1cycledrop']
steps = 100 #@param {type:"slider", min:100, max:2000, step:50}
clear_output()

seed = abs(seed)
print('Estimated Runtime: {}s.\n'.format(round(0.23*steps)+6))
!python run.py \
  -input_dir $input_dir \
  -output_dir $output_dir \
  -seed $seed \
  -noise_type $noise_type \
  -opt_name $optimizer \
  -learning_rate $learning_rate \
  -steps $steps \
  -eps $epsilon \
  -lr_schedule $learning_rate_schedule

#@markdown ---
#@markdown *If there is an error during execution or the "**Browse**" button is not active, try running this block again*

fig, (ax1, ax2) = plt.subplots(1, 2)
ax1.imshow(mpimg.imread('/content/pulse/input/face.png'))
ax1.set_title('Original')
ax2.imshow(mpimg.imread('/content/pulse/runs/face.png'))
ax2.set_title('Result')
plt.show()

In [ ]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> Download result</font></b>
try: files.download('/content/pulse/runs/face.png')
except: raise Exception("No result image")